In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.cluster

# 3.0 KL Divergence
The general idea of Variational inference is to approximate an intractable distribution with a simpler one. To achieve this, some metric is used to measure the difference between the two distributions, and some procedure to minimize this difference is applied.

The most common metric used is the Kullback-Leibler divergence (KL-divergence) and is defined as:
$$ KL(q||p) = \int q(x) \log \frac{q(x)}{p(x)} dx $$  
  
Here, we examine the KL-divergence between different distributions and see that it makes sense with our intuition of which distributions should be similar compared to the Euclidean norm.

## 3.0.1 KL-divergence and Euclidean norm functions and tests

In [ ]:
import scipy.integrate as sp_int


def KL_divergence(p_1, p_2, a=-np.inf, b=np.inf):
    D_KL, err = sp_int.quad(lambda x: p_1(x) * np.log(p_1(x)/p_2(x)), a, b)
    return D_KL

def Euclidean_norm(p_1, p_2, a, b):
    D_Euc, err = np.sqrt(sp_int.quad(lambda x: (p_1(x) - p_2(x))**2, a, b))
    return D_Euc


# Test our functions - e.g. when the two distributions are the same distances should be 0
mu_1_test = 0
mu_2_test = 0
sigma_1_test = 1
sigma_2_test = 1

p_1_test = lambda x: 1/(np.sqrt(2*np.pi) * sigma_1_test) * np.exp(-(x-mu_1_test)**2/(2 * sigma_1_test))
p_2_test = lambda x: 1/(np.sqrt(2*np.pi) * sigma_2_test) * np.exp(-(x-mu_2_test)**2/(2 * sigma_2_test))

D_KL = KL_divergence(p_1_test, p_2_test, -5, 5)  # Evaluate on [-5, 5] instead of [-inf, inf] for stability reasons
D_Euclidean = Euclidean_norm(p_1_test, p_2_test, -5, 5)

print(f"KL-divergence: {D_KL}")
print(f"Euclidean norm: {D_Euclidean}")

## 3.0.2 Plot the two distributions and compare KL-divergence and Euclidean norm

Given two Gamma distributions, we plot them and compare the KL-divergence and Euclidean norm between them.

In [ ]:
import scipy.special as sp_spec

def pdf_gamma(alpha, beta):
    # Pdfs for Gamma distributions (check Wikipedia)
    return lambda x: beta**alpha / (sp_spec.gamma(alpha)) * x**(alpha - 1) * np.exp(-beta * x)

def plot_two_pdfs(p_1, p_2, a, b):
    x = np.linspace(a, b, 100)
    y1 = p_1(x)
    y2 = p_2(x)

    plt.plot(x, y1, label=r'$p_1(x)$')
    plt.plot(x, y2, label=r'$p_2(x)$')
    plt.legend(loc='best')

In [ ]:
# Case 1
alpha_1 = 2
beta_1 = 4
alpha_2 = 2
beta_2 = 3


p_1 = pdf_gamma(alpha_1, beta_1)
p_2 = pdf_gamma(alpha_2, beta_2)

interval_a = 0
interval_b = 3
plot_two_pdfs(p_1, p_2, interval_a, interval_b)
plt.show()

D_KL_case1 = KL_divergence(p_1, p_2, interval_a, interval_b)
D_Euclidean_case1 = Euclidean_norm(p_1, p_2, interval_a, interval_b)

print(f"KL-divergence: {D_KL_case1}")
print(f"Euclidean norm: {D_Euclidean_case1}")

# Case 2
alpha_1 = 100
beta_1 = 100
alpha_2 = 100
beta_2 = 98

p_1 = pdf_gamma(alpha_1, beta_1)
p_2 = pdf_gamma(alpha_2, beta_2)

plot_two_pdfs(p_1, p_2, interval_a, interval_b)
plt.show()

D_KL_case2 = KL_divergence(p_1, p_2, interval_a, interval_b)
D_Euclidean_case2 = Euclidean_norm(p_1, p_2, interval_a, interval_b)

print(f"KL-divergence: {D_KL_case2}")
print(f"Euclidean norm: {D_Euclidean_case2}")


In [ ]:
relative_KL = D_KL_case1/D_KL_case2
relative_Euclidean = D_Euclidean_case1/D_Euclidean_case2

print(f"Relative KL-divergence: {relative_KL}")
print(f"Relative Euclidean norm: {relative_Euclidean}")

## 3.0.3 Discussion

The Euclidean norm suggests that the differences between the distributions in the two cases are similar, which is not in line with our intuition as the case 2 distributions are more similar.

The KL-divergence suggest an almost 4 times larger difference between the distributions in case 1 compared to case 2, which better captures our intuition.

# 3.1 Gaussian Mixture Model (GMM)

This workshops revolves around implementing the CAVI algorithm for a Gaussian Mixture Model (GMM), similar to the model which was introduced in the video lectures of Module 3, but with the simplification that $\mu_k$ is Normal distributed and $\tau_k$ is Gamma distributed, instead of $\mu_k, \tau_k$ being jointly Normal-Gamma distributed.



### 3.1.1.2 Priors

The priors for the GMM model are.

Component means prior:
$$ p(\mu_k) = \mathcal{N}(\mu_k | \nu_0, \lambda_0) $$
Component precisions prior:
$$ p(\tau_k) = \text{Gamma}(\tau_k | \alpha_0, \beta_0) $$

Class variable prior:
$$ p(Z | \pi) = \prod_{n=1}^N \text{Categorical}(\pi) $$

Mixture weights prior:
$$ p(\pi) = \text{Dir}(\pi | \delta_0) $$


## 3.1.1 Model description and Bayesian Network

Always start by defining the model and the Bayesian Network to get an overview of the problem.

### 3.1.1.1 Observation model
The likelihood for the GMM model is defined as:
$$ p(X_n=x_n | \mu, \tau) = \sum_{k=1}^K \pi_k \mathcal{N}(x_n | \mu_k, \tau_k) \qquad (1)$$

Instead, we will work with the model on latent variable form as (as described in Bishop 9.2):
$$ p(X_n=x_n | Z_n, \mu, \tau) = \prod_{k=1}^K \mathcal{N}(x_n | \mu_k, \tau_k)^{I(Z_n = k)} \qquad (2)$$

which is an easier form to work with. $(1)$ and $(2)$ are equivalent when $(2)$ is marginalized over $Z_n$, i.e., 
$p(X_n=x_n | \mu, \tau) = \sum_k p(X_n=x_n | Z_n=k, \mu_k, \tau_k)p(Z_n=k | \pi)$.

### 3.1.1.3 Bayesian Network

Write out the Bayesian Network for the model described above.

Draw on board.

## 3.1.2 Generate data
Next we generate some data to work with. This is also a good way to get a better understanding of the model and the problem.



In [ ]:
def generate_data(N, K, nu_0, lambda_0, alpha_0, beta_0, delta_0):
    return X, Z_true, pi_true, mu_true, tau_true

def generate_pi(delta_0):
    pass

def generate_mu(nu_0, lambda_0, K):
    pass

def generate_tau(a_0, b_0, K):
    pass

def generate_Z(N, pi_true):
    pass

def generate_X(N, mu_true, tau_true, Z):
    pass
    

### Test the data generation functions

In [ ]:
K_test = 2
N_test = 100

# Fix mu_1 and mu_2 and tau_1 and tau_2
mu_1_test = 100
mu_2_test = 1
mu_test = [mu_1_test, mu_2_test]
tau_test = [1, 1]

# Fix pi - biased towards component 1
pi_test = np.array([0.8, 0.2])

Z_test = generate_Z(N_test, pi_test)
X_test = generate_X(N_test, mu_test, tau_test, Z_test)

print(f"X mean: {X_test.mean()}")
assert X_test.mean() > 60.0  # Expect the mean of X to be much closer to mu_1 than mu_2
Z_test_one_hot = np.eye(K_test)[Z_test]
print(f"Z: {Z_test_one_hot.mean(axis=0)}")
assert np.allclose(Z_test_one_hot.mean(axis=0), pi_test, atol=0.1)  # Expect the mean of Z to be close to pi

## 3.1.3 Coordinate ascent VI (CAVI)

General algorithm for CAVI for the GMM:
1. Initialize the variational parameters $\gamma$.
2. While the ELBO has not converged:
    3. Update the variational parameters for each latent variable $z_n$.
    4. Update the variational parameters for each component mean $\mu_k$.
    5. Update the variational parameters for each component precision $\tau_k$.
    6. Update the variational parameters for the class variable $\pi$.
    7. Calculate the ELBO. (To measure convergence).


### CAVI updates

Derivation on board, and taken from the video lectures.

When deriving the CAVI updates, always begin with writing out the factorization of the joint:

$$ p(X, Z, \mu, \tau, \pi) = p(X | Z, \mu, \tau) p(Z | \pi) p(\mu) p(\tau) p(\pi) $$

And the minimum mean-field approximation:
$$ q(Z, \mu, \tau, \pi) = q(Z)q(\pi, \mu, \tau)$$

Which, in video lectures, are shown to simplify further to: 
$$q(Z)q(\pi, \mu, \tau) = q(Z) q(\pi) \prod_k q(\mu_k) q(\tau_k) $$

Then apply the CAVI update equations for each variational distribution. Let's start with $q(\mu_k)$:

$$ \log q^*(\mu_k) \stackrel{+}{=} \mathbb{E}_{q(Z, \mu_{\neg k}, \tau, \pi)}\big[\log p(X, Z, \mu, \tau, \pi)\big] $$

Derive on board.

### Fixed updates

## 3.1.4 Running the CAVI algorithm on simulated data

In [ ]:
def CAVI_update_mu_k(x, q_Z_k, E_tau_k, nu_0, lambda_0):
    """
    Implement this based on the CAVI update derived on the board.
    """
    return nu_star, lambda_star


# Updates based on the video lectures (slight modifications)
def CAVI_update_Z(E_log_tau, E_tau, E_x_mu2, E_log_pi):
    return rho_star


def CAVI_update_tau(q_Z, alpha_0, beta_0, E_x_mu2):
    return alpha_star, beta_star


def CAVI_update_pi(q_Z, delta_0):
    return delta_star


def calculate_ELBO(x, r_q, nu_q, lambda_q, alpha_q, beta_q, delta_q, prior_params):
    nu_0, lambda_0, alpha_0, beta_0, delta = prior_params
    N, K = x.shape[0], nu_q.shape[0]

    E_tau = alpha_q / beta_q
    E_log_tau = sp_spec.digamma(alpha_q) - np.log(beta_q)
    E_log_pi = sp_spec.digamma(delta_q) - sp_spec.digamma(np.sum(delta_q))
    E_mu = nu_q
    E_mu2 = nu_q ** 2 + 1 / lambda_q ** 2

    x_n2_rnk_sum_n = np.einsum('n, nk->k', x ** 2, r_q)
    x_n_rnk_sum_n = np.einsum('n, nk->k', x, r_q)
    rnk_sum_n = np.einsum('nk->k', r_q)

    ELBO_p_X_Z_mu_tau = 1/2 * np.einsum('k, k->', E_log_tau, np.sum(r_q, axis=0)) - N/2 * np.log(2 * np.pi) - \
                1/2 * np.einsum('k, k->', E_tau, x_n2_rnk_sum_n - 2 * x_n_rnk_sum_n * E_mu + E_mu2 * rnk_sum_n)

    ELBO_p_Z_pi = np.einsum('nk,k->', r_q, E_log_pi)
    ELBO_p_mu = np.einsum('k->', 1/2 * np.log(lambda_0) - 1/2 * np.log(2*np.pi)
                          - 1/2 * lambda_0 * (E_mu2 - 2*E_mu*nu_0 + nu_0**2))
    ELBO_p_tau = np.einsum('k->', alpha_0*np.log(beta_0) - sp_spec.gammaln(alpha_0)
                            + (alpha_0 - 1)*E_log_tau - beta_0*E_tau)
    log_Beta_func_delta_0 = sp_spec.gammaln(delta).sum() - sp_spec.gammaln(np.sum(delta))
    ELBO_p_pi = np.einsum('k->', (delta - 1) * E_log_pi) - log_Beta_func_delta_0

    H_qZ = -np.einsum('nk->', r_q * np.log(r_q))
    H_qmu = 1/2 * np.einsum('k->', -np.log(lambda_q) + 1 + np.log(2*np.pi))
    H_qtau = np.einsum('k->', sp_spec.gammaln(alpha_q) - (alpha_q - 1)*sp_spec.psi(alpha_q) - np.log(beta_q)
                        + alpha_q)
    log_Beta_func_delta_q = sp_spec.gammaln(delta_q).sum() - sp_spec.gammaln(np.sum(delta_q))
    delta_q_sum = np.sum(delta_q)
    H_qpi = log_Beta_func_delta_q + (delta_q_sum - K)*sp_spec.digamma(delta_q_sum) - ((delta_q - 1)*sp_spec.digamma(delta_q)).sum()

    ELBO = ELBO_p_X_Z_mu_tau + ELBO_p_Z_pi + ELBO_p_mu + ELBO_p_tau + ELBO_p_pi + H_qZ + H_qmu + H_qtau + H_qpi
    return ELBO


def CAVI_algorithm(x, K, prior_params, max_iter=100, tol=1e-3, step_size=0.01):
    N = x.shape[0]
    nu_0, lambda_0, alpha_0, beta_0, delta_0 = prior_params

    # Define Variational parameters
    I = max_iter + 1
    r_q = np.zeros((I, N, K))                               # N x K (params for q(Z))
    alpha_q, beta_q = np.zeros((I, K)), np.zeros((I, K))    # K     (params for q(tau))
    nu_q, lambda_q = np.zeros((I, K)), np.zeros((I, K))     # K     (params for q(mu))
    delta_q = np.zeros((I, K))                              # K     (params for q(pi))

    # Define Expected Values functions
    E_tau_map = lambda alpha, beta: alpha / beta
    E_log_tau_map = lambda alpha, beta: sp_spec.psi(alpha) - np.log(beta)
    E_log_pi_map = lambda delta: sp_spec.psi(delta) - sp_spec.psi(np.sum(delta))
    E_mu_map = lambda nu: nu
    E_mu2_map = lambda nu, lmbda: nu ** 2 + 1 / lmbda ** 2
    E_x_mu2_map = lambda x, nu, lmbda: ((np.einsum('n,n,k->nk', x, x, np.ones(K))
                                        - 2 * np.einsum('n, k -> nk', x, E_mu_map(nu)))
                                        + np.einsum('k,n->nk', E_mu2_map(nu, lmbda), np.ones(N)))

    # Initialize the variational parameters
    labels = sklearn.cluster.KMeans(n_clusters=K).fit(x.reshape(-1, 1)).labels_
    delta_q[0] = delta_0
    alpha_q[0], beta_q[0] = alpha_0, beta_0
    lambda_q[0] = lambda_0
    for k in range(K):
        x_k = x[labels == k]
        nu_q[0, k] = np.mean(x_k)

    # Initialize expected values
    E_tau = E_tau_map(alpha_q[0], beta_q[0])
    E_log_tau = E_log_tau_map(alpha_q[0], beta_q[0])
    E_log_pi = E_log_pi_map(delta_q[0])
    E_x_mu2 = E_x_mu2_map(x, nu_q[0], lambda_q[0])  # N x K

    ELBO = np.zeros(max_iter)
    
    
    for i in range(max_iter):
        """
        Implement the for loop of the CAVI algorithm here.
        """

    out = {'ELBO': ELBO, 'r_q': r_q, 'nu_q': nu_q, 'lambda_q': lambda_q,
           'alpha_q': alpha_q, 'beta_q': beta_q, 'delta_q': delta_q}
    return out


### Generate data

In [ ]:
N_sim = 1000
K_sim = 3
mu_true = np.array([1, 3, 6])
tau_true = np.array([5, 5, 5])
pi_true = np.array([0.2, 0.3, 0.5])
Z_true = generate_Z(N_sim, pi_true)

x = generate_X(N_sim, mu_true, tau_true, Z_true)


In [ ]:
# Histogram of the data
plt.hist(x, bins=30)
plt.xlabel('x')
plt.ylabel('Number of samples')
plt.show()

### Run CAVI

In [ ]:
# Algorithm parameters
np.random.seed(0)
max_iter = 1000
tol = 1e-3
K = 3
N = x.shape[0]
step_size = 0.1

# Prior parameters
nu_0 = np.array([1., 1., 1.]) * 2.
lambda_0 = 10.
alpha_0 = 1.
beta_0 = 1.
delta_0 = np.ones(K) * N/K
prior_params = (nu_0, lambda_0, alpha_0, beta_0, delta_0)

# Run CAVI
out = CAVI_algorithm(x, K, prior_params, max_iter, tol, step_size=step_size)


### Plot the ELBO

In [ ]:
plt.plot(out['ELBO'])
plt.xlabel('Iteration')
plt.ylabel('ELBO')
plt.title('ELBO over iterations')

### Visualize the model fit

In [ ]:
import scipy.stats as sp_stats
x_axis = np.linspace(-2, 15, 1000)
plt.hist(x, bins=30, density=True, alpha=0.5, label='Data')
for k in range(K):
    mu_MAP_k = out['nu_q'][-1][k]
    tau_MAP_k = out['alpha_q'][-1][k] / out['beta_q'][-1][k]
    pi_MAP_k = out['delta_q'][-1][k] / np.sum(out['delta_q'][-1])
    plt.plot(x_axis, pi_MAP_k * sp_stats.norm.pdf(x_axis, mu_MAP_k, 1 / np.sqrt(tau_MAP_k)),
             label=f'Component {k + 1}')

### Plot parameters trajectories

In [ ]:
plt.plot(out['alpha_q'], out['beta_q'])
plt.xlabel('alpha_q')
plt.ylabel('beta_q')
plt.title('alpha_q and beta_q trajectories')

### Label switching

The labels of the clusters inferred by the algorithm can be permuted compared to the true labels. This is known as the label switching problem. One way to handle this is to find the permutation of the inferred labels that best matches the true labels.

In [ ]:
print(f"True pi: {pi_true}")
print(f"E_q[pi]: {out['delta_q'][-1]/np.sum(out['delta_q'][-1])}")
print(f"True mu: {mu_true}")
print(f"E_q[mu]: {out['nu_q'][-1]}")